# Worksheet 08

Name:  Annie Liang

UID:  U44666626

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [191]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.666235391659105, 5.616372601992778, 4.968628620102518, 5.443542323074169, 5.923283744102789, 3.250383095576413, 5.36707311033284, 5.576107067034092, 7.255641740484222, 3.979967973813479]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [192]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[7.584911316233825, 9.813546922368571, 8.918232243470554, 9.39913810821606, 6.111311625269099, 5.602154720133019, 8.57338494417693, 10.215997245053165, 8.461079469645696, 7.102327040870739]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [193]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[3.979967973813479, 7.102327040870739, 8.461079469645696, 10.215997245053165, 8.57338494417693, 7.255641740484222, 5.576107067034092, 5.36707311033284, 3.250383095576413, 5.923283744102789]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

- Prior Probability of s1 = 0.5
- Mean of s1 = 5
- Variance of s1 = 1

- Prior Probability of s2 = 0.5
- Mean of s2 = 8
- Variance of s2 = 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [194]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
#Take everything in data that was assigned to s1
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[3.979967973813479, 5.576107067034092, 5.36707311033284, 3.250383095576413, 5.923283744102789]
[7.102327040870739, 8.461079469645696, 10.215997245053165, 8.57338494417693, 7.255641740484222]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.819362998171923,  mean_2 = 8.32168608804615
var_1 = 1.051514181536857,  var_2 = 1.2588969250939637


/Users/annieliang/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The estimates are close to the true values, although there is a slight discrepancy in the estimation of the second variance. The estimated probability matches for both, both the means are relatively close to their respective means, and both the variance are relatively close to their respective variance with the second variance a little more off.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [195]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  3.979967973813479
probability of observing that point if it came from cluster 0 =  0.2758799815417015
probability of observing that point if it came from cluster 1 =  0.000828097735730578
point =  7.102327040870739
probability of observing that point if it came from cluster 0 =  0.035934697492192025
probability of observing that point if it came from cluster 1 =  0.19824317537153585
point =  8.461079469645696
probability of observing that point if it came from cluster 0 =  0.0009430251780634244
probability of observing that point if it came from cluster 1 =  0.31496158344837
point =  10.215997245053165
probability of observing that point if it came from cluster 0 =  7.234982998559084e-07
probability of observing that point if it came from cluster 1 =  0.10215193936878326
point =  8.57338494417693
probability of observing that point if it came from cluster 0 =  0.0006477442251668433
probability of observing that point if it came from cluster 1 =  0.31062725489435955
point =  7.

In general, these estimations are good with points that are closer to 5 have higher estimated probabilities of being from s1 and points that are closer to 8 have higher probabilities of being from s2.

Points for s1: 3.97, 5.57, 5.36, 3.25, 5.92

Points for s2: 7.10, 8.46, 10.21, 8.57, 7.25

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [196]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.883732506744383,  mean_2 = 8.204168802669145
var_1 = 1.3052285450328416,  var_2 = 1.6220093431878824


The estimated mean for both means have slightly improved, while both variances deviated further from the actual variance (both increased). However, our estimates would most likely benefit from further iterations.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [197]:

from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)

prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x)]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]
var = [sum([x[0] * (x[1] - mean[0]) ** 2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * (x[1] - mean[1]) ** 2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  3.979967973813479
probability of observing that point if it came from cluster 0 =  0.24049921496023616
probability of observing that point if it came from cluster 1 =  0.00828095864405963
point =  7.102327040870739
probability of observing that point if it came from cluster 0 =  0.07208323379471078
probability of observing that point if it came from cluster 1 =  0.19527762333560436
point =  8.461079469645696
probability of observing that point if it came from cluster 0 =  0.007145590667633432
probability of observing that point if it came from cluster 1 =  0.24288966202428824
point =  10.215997245053165
probability of observing that point if it came from cluster 0 =  7.262507454483248e-05
probability of observing that point if it came from cluster 1 =  0.1139706041309844
point =  8.57338494417693
probability of observing that point if it came from cluster 0 =  0.005623584820673123
probability of observing that point if it came from cluster 1 =  0.23966535120525262
point =  7.2

These estimated probabilities seem to be more reasonable with proababolities values below 6.5 have a higher probability to s1 and values above 6.5 have a higher probability to s2.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [198]:
assignments = {}
for p in range(len(data)):
    if prob_s0_x[p] > prob_s1_x[p]:
        assignments[data[p]] = 0
    else:
        assignments[data[p]] = 1
    print("Point ", data[p], " assigned to ", assignments[data[p]])
    print()

Point  3.979967973813479  assigned to  0

Point  7.102327040870739  assigned to  1

Point  8.461079469645696  assigned to  1

Point  10.215997245053165  assigned to  1

Point  8.57338494417693  assigned to  1

Point  7.255641740484222  assigned to  1

Point  5.576107067034092  assigned to  0

Point  5.36707311033284  assigned to  0

Point  3.250383095576413  assigned to  0

Point  5.923283744102789  assigned to  0

